In [3]:
import pandas as pd

In [4]:
df_posts = pd.read_csv('./posts_predicted.csv')

/var/folders/1v/mg6_xn492j3g8l1qd503m2n80000gn/T/ipykernel_70752/1247585743.py:1: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_posts = pd.read_csv('./posts_predicted.csv')


In [7]:
df_replies = df_posts[~df_posts.is_parent_post.astype(bool)]

In [8]:
df_replies.shape

(2914128, 30)

In [9]:
df_replies['token_num'] = df_replies.comment_text.fillna('').str.split(' ').apply(lambda x: len(x))

/var/folders/1v/mg6_xn492j3g8l1qd503m2n80000gn/T/ipykernel_70752/2691881560.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_replies['token_num'] = df_replies.comment_text.fillna('').str.split(' ').apply(lambda x: len(x))


In [26]:
df_replies_filtered = df_replies[
    df_replies['token_num'] >= 10
]

In [27]:
flags = ['TOXICITY', 'IDENTITY_ATTACK', 'INSULT', 'PROFANITY', 'THREAT', 'SEXUALLY_EXPLICIT']

# 1: safe
# 0: unsafe
df_replies_filtered['safety_score'] = 1 - df_replies_filtered[flags].mean(axis=1)

/var/folders/1v/mg6_xn492j3g8l1qd503m2n80000gn/T/ipykernel_70752/4050337512.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_replies_filtered['safety_score'] = 1 - df_replies_filtered[flags].mean(axis=1)


In [35]:
res = pd.DataFrame()

for flag in flags:
    data_certain = df_replies_filtered[df_replies_filtered[flag] > 0.6]
    data_certain['type'] = 'unsafe'
    data_marginal = df_replies_filtered[
        (df_replies_filtered[flag] >= 0.4) &
        (df_replies_filtered[flag] <= 0.6)
    ]
    data_marginal['type'] = 'marginal'

    data_slight = df_replies_filtered[
        (df_replies_filtered[flag] >= 0.25) &
        (df_replies_filtered[flag] < 0.4)
    ]

    data_slight['type'] = 'slight'

    res = pd.concat([
        res, data_certain, data_marginal, data_slight
    ])

res = res.drop_duplicates('id')
    

/var/folders/1v/mg6_xn492j3g8l1qd503m2n80000gn/T/ipykernel_70752/889736065.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_certain['type'] = 'unsafe'
/var/folders/1v/mg6_xn492j3g8l1qd503m2n80000gn/T/ipykernel_70752/889736065.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_marginal['type'] = 'marginal'
/var/folders/1v/mg6_xn492j3g8l1qd503m2n80000gn/T/ipykernel_70752/889736065.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

In [36]:
res.type.value_counts()

type
slight      8918
marginal    2691
unsafe      1051
Name: count, dtype: int64

In [38]:
res_unsafe = res[res.type != 'slight']
res_slight = res[res.type == 'slight']

res_enhanced = pd.concat([
    res_unsafe, res_slight.sample(2000)
])

In [39]:
res_enhanced.type.value_counts()

type
marginal    2691
slight      2000
unsafe      1051
Name: count, dtype: int64

In [40]:
res_enhanced.comment_text

8143       Gonna shoot myself I shoulder known how to do ...
11047      Mohamad shut your mouth please I don't like be...
11695      i completly agree with you daniel, now go to h...
11699      i completly agree with you daniel, now go to h...
17101      Oh go well now that I look at James' answer I ...
                                 ...                        
1401780    XD! He needs one if he was here right now I be...
1989492    After that you would just square them...meanin...
2482076    I will be honest with you, anime boys do fiddl...
1404139    I messed up ok but everbody makes mistakes yal...
1364512    Alexandra will you go out with me? Your math s...
Name: comment_text, Length: 5742, dtype: object

In [33]:
res[['id', 'useraccount_id', 'comment_text', 'type']].shape

(12660, 4)

In [41]:
res_enhanced.to_csv('./mturk_sample.csv', index=False)